# SLR 10X correlation and comparison

In [1]:
!date

Tue Aug 20 11:56:18 PDT 2024


In [2]:
!echo $CONDA_PREFIX

/c4/home/derek/miniconda3/envs/scquint_4


In [3]:
import pandas as pd
import numpy as np
import scanpy as sc
from scipy import stats

import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
#adata_10X = sc.read_h5ad('/c4/home/derek/data1/HDD-2/derek/SLR_temp/CAT_20220904_PBMC/concat.3.new.h5ad')

In [5]:
## cellranger input
adata_10X = sc.read_10x_h5('/c4/home/derek/data1/derek/data_scSLR/PBMC/10X/cellranger_outs/PBMC_sample_1/outs/raw_feature_bc_matrix.h5')

/c4/home/derek/miniconda3/envs/scquint_4/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [6]:
## STAR solo input
adata_10X = sc.read_mtx('/c4/home/derek/data1/derek/data_scSLR/PBMC/10X/STAR_outs/sample_2/StarOut_10X/Solo.out/GeneFull/raw/matrix.mtx').T

adata_10X.var_names = pd.read_csv('/c4/home/derek/data1/derek/data_scSLR/PBMC/10X/STAR_outs/sample_2/StarOut_10X/Solo.out/GeneFull/raw/features.tsv',
            sep='\t',header=None)[1]

adata_10X.obs_names = pd.read_csv('/c4/home/derek/data1/derek/data_scSLR/PBMC/10X/STAR_outs/sample_2/StarOut_10X/Solo.out/GeneFull/raw/barcodes.tsv',
            sep='\t',header=None)[0]

In [7]:
adata_10X

AnnData object with n_obs × n_vars = 737280 × 36385

In [8]:
# adata_10X = adata_10X[:,adata_10X.var['feature_types'] == 'Gene Expression']

In [9]:
adata_SLR = sc.read_mtx('/c4/home/derek/data1/derek/data_scSLR/PBMC/STAR_outs/sample_2/StarOut_5prime/Solo.out/GeneFull/raw/matrix.mtx').T

adata_SLR.var_names = pd.read_csv('/c4/home/derek/data1/derek/data_scSLR/PBMC/STAR_outs/sample_2/StarOut_5prime/Solo.out/GeneFull/raw/features.tsv',
            sep='\t',header=None)[1]

adata_SLR.obs_names = pd.read_csv('/c4/home/derek/data1/derek/data_scSLR/PBMC/STAR_outs/sample_2/StarOut_5prime/Solo.out/GeneFull/raw/barcodes.tsv',
            sep='\t',header=None)[0]

In [10]:
adata_10X.obs_names

Index(['AAACCTGAGAAACCAT', 'AAACCTGAGAAACCGC', 'AAACCTGAGAAACCTA',
       'AAACCTGAGAAACGAG', 'AAACCTGAGAAACGCC', 'AAACCTGAGAAAGTGG',
       'AAACCTGAGAACAACT', 'AAACCTGAGAACAATC', 'AAACCTGAGAACTCGG',
       'AAACCTGAGAACTGTA',
       ...
       'TTTGTCATCTTGGGTA', 'TTTGTCATCTTGTACT', 'TTTGTCATCTTGTATC',
       'TTTGTCATCTTGTCAT', 'TTTGTCATCTTGTTTG', 'TTTGTCATCTTTACAC',
       'TTTGTCATCTTTACGT', 'TTTGTCATCTTTAGGG', 'TTTGTCATCTTTAGTC',
       'TTTGTCATCTTTCCTC'],
      dtype='object', length=737280)

In [11]:
adata_SLR.obs_names

Index(['AAACCTGAGAAACCAT', 'AAACCTGAGAAACCGC', 'AAACCTGAGAAACCTA',
       'AAACCTGAGAAACGAG', 'AAACCTGAGAAACGCC', 'AAACCTGAGAAAGTGG',
       'AAACCTGAGAACAACT', 'AAACCTGAGAACAATC', 'AAACCTGAGAACTCGG',
       'AAACCTGAGAACTGTA',
       ...
       'TTTGTCATCTTGGGTA', 'TTTGTCATCTTGTACT', 'TTTGTCATCTTGTATC',
       'TTTGTCATCTTGTCAT', 'TTTGTCATCTTGTTTG', 'TTTGTCATCTTTACAC',
       'TTTGTCATCTTTACGT', 'TTTGTCATCTTTAGGG', 'TTTGTCATCTTTAGTC',
       'TTTGTCATCTTTCCTC'],
      dtype='object', length=737280)

In [12]:
#adata_10X = adata_10X[adata_10X.obs_names.str.endswith('-1-22')]
#adata_10X.obs_names = adata_10X.obs_names.str[:-2]

In [13]:
adata_SLR= adata_SLR[adata_SLR.obs_names.isin(adata_10X.obs_names)]

In [14]:
adata_10X.var_names_make_unique()
adata_SLR.var_names_make_unique()

/c4/home/derek/miniconda3/envs/scquint_4/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [19]:
adata_SLR = adata_SLR[:,adata_SLR.var_names.isin(adata_10X.var_names)]
adata_10X = adata_10X[:,adata_10X.var_names.isin(adata_SLR.var_names)]

In [20]:
# filter cells by name

cells_used = pd.read_csv('/c4/home/derek/data1/derek/data_scSLR/PBMC/PBMC_sample_2_idents.csv',header=None,sep='\t')[0]

adata_SLR = adata_SLR[adata_SLR.obs_names.isin(cells_used)]
adata_10X = adata_10X[adata_10X.obs_names.isin(cells_used)]

In [21]:
sc.pp.calculate_qc_metrics(adata_10X,inplace=True)
sc.pp.calculate_qc_metrics(adata_SLR,inplace=True)

/c4/home/derek/miniconda3/envs/scquint_4/lib/python3.10/site-packages/scanpy/preprocessing/_qc.py:135: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs[obs_metrics.columns] = obs_metrics
/c4/home/derek/miniconda3/envs/scquint_4/lib/python3.10/site-packages/scanpy/preprocessing/_qc.py:135: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs[obs_metrics.columns] = obs_metrics


In [22]:
adata_10X.X.sum(axis=0)

matrix([[1., 2., 0., ..., 0., 5., 0.]], dtype=float32)

In [31]:
comp = pd.concat([adata_10X.obs.n_genes_by_counts, adata_SLR.obs.n_genes_by_counts],axis=1)
comp.columns = ['10X','scSLR']

In [32]:
comp

,10X,scSLR
AAACCTGAGAGTACAT,2916,2007
AAACCTGAGAGTGACC,1468,933
AAACCTGAGATGTTAG,2263,1498
AAACCTGAGCAGCCTC,1371,896
AAACCTGAGCGTTCCG,1090,676
...,...,...
TTTGTCAGTGTTGAGG,1546,998
TTTGTCAGTTGGACCC,2486,1660
TTTGTCAGTTGTACAC,1265,832
TTTGTCATCCGGGTGT,1190,777


# Plot correlation between library types

In [ ]:
values = np.vstack([adata_SLR.var.log1p_total_counts,
                    adata_10X.var.log1p_total_counts])
kernel = stats.gaussian_kde(values)(values)


In [ ]:
corr = stats.pearsonr(x=adata_SLR.var.log1p_total_counts,
            y=adata_10X.var.log1p_total_counts,)

In [ ]:
corr

In [ ]:
%matplotlib inline

fig, ax = plt.subplots(figsize=(6, 6))

n = sns.scatterplot(x=values[1],
            y=values[0],
            s=10,
            linewidth=0,
            hue=kernel,
            palette='plasma',
            ax=ax,
            hue_norm = (0,0.1)
           )

ax.get_legend().remove()
# ax.figure.colorbar(n)

ax.spines.right.set_visible(False)
ax.spines.top.set_visible(False)

ax.spines.bottom.set_linewidth(2)
ax.spines.left.set_linewidth(2)

ax.set_title(f'r = {round(corr[0],4)}',loc='left',fontsize=16)

ax.set_xlabel("log(total counts) 10X 5'", fontsize=16)
ax.set_ylabel("log(total counts) scSLR-seq 5'", fontsize=16)

ax.tick_params(axis='both', which='major', labelsize=12)
# fig.savefig('./plots/SLR_10X_corr_PBMC.pdf')

# Plot correlation between SLR read types

In [ ]:
adata_SLR_int = sc.read_mtx('/c4/home/derek/data1/derek/data_scSLR/PBMC/STAR_outs/sample_2/StarOut_internal/Solo.out/GeneFull/raw/matrix.mtx').T

adata_SLR_int.var_names = pd.read_csv('/c4/home/derek/data1/derek/data_scSLR/PBMC/STAR_outs/sample_2/StarOut_internal/Solo.out/GeneFull/raw/features.tsv',
            sep='\t',header=None)[1]

adata_SLR_int.obs_names = pd.read_csv('/c4/home/derek/data1/derek/data_scSLR/PBMC/STAR_outs/sample_2/StarOut_internal/Solo.out/GeneFull/raw/barcodes.tsv',
            sep='\t',header=None)[0]

In [ ]:
adata_SLR_int.var_names_make_unique()

In [ ]:
adata_SLR_int = adata_SLR_int[:,adata_SLR_int.var_names.isin(adata_SLR.var_names)]
adata_SLR_5p = adata_SLR[:,adata_SLR.var_names.isin(adata_SLR_int.var_names)]



In [ ]:
sc.pp.calculate_qc_metrics(adata_SLR_int,inplace=True)
sc.pp.calculate_qc_metrics(adata_SLR_5p,inplace=True)

In [ ]:
values = np.vstack([adata_SLR_5p.var.log1p_total_counts,
                    adata_SLR_int.var.log1p_total_counts])
kernel = stats.gaussian_kde(values)(values)


In [ ]:
corr = stats.pearsonr(x=adata_SLR_5p.var.log1p_total_counts,
            y=adata_SLR_int.var.log1p_total_counts,)

In [ ]:
corr

In [ ]:
%matplotlib inline

fig, ax = plt.subplots(figsize=(6, 6))

n = sns.scatterplot(x=values[1],
            y=values[0],
            s=10,
            linewidth=0,
            hue=kernel,
            palette='plasma',
            ax=ax,
            hue_norm = (0,0.1)
           )

ax.get_legend().remove()
# ax.figure.colorbar(n)

ax.spines.right.set_visible(False)
ax.spines.top.set_visible(False)

ax.spines.bottom.set_linewidth(2)
ax.spines.left.set_linewidth(2)

ax.set_title(f'r = {round(corr[0],4)}',loc='left',fontsize=16)

ax.set_xlabel("log(total counts) scSLR-seq 5'", fontsize=16)
ax.set_ylabel('log(total counts) scSLR-seq internal', fontsize=16)

ax.tick_params(axis='both', which='major', labelsize=12)
# fig.savefig('./plots/SLR_lib_portion_corr_PBMC.pdf')

# Plot correlation between biological replicates

In [ ]:
adata_SLR_s1 = sc.read_mtx('/c4/home/derek/data1/derek/data_scSLR/PBMC/STAR_outs/sample_1/StarOut_5prime/Solo.out/GeneFull/raw/matrix.mtx').T

adata_SLR_s1.var_names = pd.read_csv('/c4/home/derek/data1/derek/data_scSLR/PBMC/STAR_outs/sample_1/StarOut_5prime/Solo.out/GeneFull/raw/features.tsv',
            sep='\t',header=None)[1]

adata_SLR_s1.obs_names = pd.read_csv('/c4/home/derek/data1/derek/data_scSLR/PBMC/STAR_outs/sample_1/StarOut_5prime/Solo.out/GeneFull/raw/barcodes.tsv',
            sep='\t',header=None)[0]

In [ ]:
adata_SLR_s1.var_names_make_unique()

In [ ]:
adata_SLR_s1 = adata_SLR_s1[:,adata_SLR_s1.var_names.isin(adata_SLR.var_names)]
adata_SLR_s2 = adata_SLR[:,adata_SLR.var_names.isin(adata_SLR_s1.var_names)]


In [ ]:
sc.pp.calculate_qc_metrics(adata_SLR_s1,inplace=True)
sc.pp.calculate_qc_metrics(adata_SLR_s2,inplace=True)

In [ ]:
values = np.vstack([adata_SLR_s1.var.log1p_total_counts,
                    adata_SLR_s2.var.log1p_total_counts])
kernel = stats.gaussian_kde(values)(values)

In [ ]:
corr = stats.pearsonr(x=adata_SLR_s1.var.log1p_total_counts,
            y=adata_SLR_s2.var.log1p_total_counts,)

In [ ]:
corr

In [ ]:
%matplotlib inline

fig, ax = plt.subplots(figsize=(6, 6))

n = sns.scatterplot(x=values[1],
            y=values[0],
            s=10,
            linewidth=0,
            hue=kernel,
            palette='plasma',
            ax=ax,
            hue_norm = (0,0.2)
           )

ax.get_legend().remove()
# ax.figure.colorbar(n)

ax.spines.right.set_visible(False)
ax.spines.top.set_visible(False)

ax.spines.bottom.set_linewidth(2)
ax.spines.left.set_linewidth(2)

ax.set_title(f'r = {round(corr[0],4)}',loc='left',fontsize=16)

ax.set_xlabel("log(total counts) scSLR-seq 5' sample 1'", fontsize=16)
ax.set_ylabel("log(total counts) scSLR-seq 5' sample 2'", fontsize=16)

ax.tick_params(axis='both', which='major', labelsize=12)
# fig.savefig('./plots/SLR_lib_replicate_corr_PBMC.pdf')

In [ ]:
break

In [ ]:
values

In [ ]:
residual = values[1]-values[0]

In [ ]:
residual

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(4, 2),sharey=True)


n = sns.scatterplot(x=values[1],
            y=values[0],
            s=10,
            linewidth=0,
            hue=residual,
            palette='plasma',
            ax=axes[0],
           )

n = sns.scatterplot(x=values[1],
            y=values[0],
            s=10,
            linewidth=0,
            hue=np.abs(residual),
            palette='inferno',
            ax=axes[1],
           )


axes[0].get_legend().remove()
axes[1].get_legend().remove()


# ax.set_title(f'r = {round(corr[0],4)}',loc='left',fontsize=8)

# ax.set_ylabel("log(total counts) 10X 5'", fontsize=8)
# ax.set_xlabel('log(total counts) scSLR-seq', fontsize=8)

# #fig.savefig('SLR_10X_corr_PBMC.pdf')

In [ ]:
%matplotlib inline

fig, ax = plt.subplots(figsize=(5, 4))

n = sns.scatterplot(x=values[1],
            y=values[0],
            s=20,
            linewidth=0,
            hue=np.abs(residual),
            palette='inferno',
            ax=ax,
           )

# ax.get_legend().remove()
# ax.figure.colorbar(n)

ax.spines.right.set_visible(False)
ax.spines.top.set_visible(False)

# ax.spines.bottom.set_linewidth(2)
# ax.spines.left.set_linewidth(2)

# ax.set_title(f'r = {round(corr[0],4)}',loc='left')

ax.set_xlabel("log(total counts) 10X 5'", fontsize=16)
ax.set_ylabel('log(total counts) scSLR-seq', fontsize=16)


norm = plt.Normalize(residual.min(), residual.max())
sm = plt.cm.ScalarMappable(cmap="inferno", norm=norm)
sm.set_array([])

#Remove the legend and add a colorbar
ax.get_legend().remove()

ax.figure.colorbar(sm, shrink=0.4, aspect=4)



#fig.savefig('./plots/SLR_10X_corr_residuals_PBMC.pdf')

### collisions vs counts

In [ ]:
input_dir='/c4/home/derek/data1/derek/data_scSLR/PBMC/'

In [ ]:
collisions = pd.read_csv(input_dir+'sample_2_collisions_table.csv')

In [ ]:
df_residual = pd.DataFrame(residual).rename({0:'residual'},axis=1)

In [ ]:
df_residual['gene_id'] = adata_10X.var_names

In [ ]:
df_residual.sort_values(by='residual',ascending=False).head(50)

In [ ]:
collisions

In [ ]:
adata_10X.var

In [ ]:
df_residual

In [ ]:
df_residual['collision'] = df_residual.gene_id.map(dict(zip(collisions['gene_ID'],collisions['True'])))

In [ ]:
df_residual.sort_values(by = 'collision', ascending=False)

In [ ]:
sns.scatterplot(data=df_residual,
                x='collision',
                y='residual'
               )

In [ ]:
adata_SLR = adata_SLR[:,adata_SLR.var.gene_id.isin(collisions.gene_ID)]
adata_10X = adata_10X[:,adata_10X.var.gene_id.isin(collisions.gene_ID)]

In [ ]:
values = np.vstack([adata_SLR.var.log1p_total_counts,
                    adata_10X.var.log1p_total_counts])

In [ ]:
collisions = collisions.set_index('gene_name').reindex(index=adata_SLR.var_names)

In [ ]:
collisions['True_rate'].to_numpy()

In [ ]:
break

In [ ]:
#pd.DataFrame(dict(zip(adata_10X.var.gene_ids, residual)), index=[0],).T.to_csv(input_dir+'correlation_residuals.csv',header=None)

In [ ]:
res_df = pd.DataFrame(dict(zip(adata_10X.var.gene_ids, residual)), index=[0],).T
res_df.columns=['residual']

In [ ]:
res_df = res_df.sort_values(by='residual',ascending=False).reset_index().rename({'index':'gene_id'}, axis=1)

In [ ]:
res_df_ = res_df[res_df.residual >= 8]

In [ ]:
res_df_

In [ ]:
from pybiomart import Dataset

dataset = Dataset(name='hsapiens_gene_ensembl',
                  host='http://www.ensembl.org')

genes = dataset.query(attributes=['ensembl_gene_id','external_gene_name','chromosome_name','start_position','end_position'],
              )

In [ ]:
res_df_.gene_id.map(dict(zip(genes['Gene stable ID'],genes['Gene name']))).head(60)

In [ ]:
echo !conda list